## Query GPT-4 for name and analysis using a toy example

#### This uses an improved version of the original prompt that includes instructions to generate an LLM Confidence Score.

#### The prompt also includes an example analysis to help the LLM in its task.

#### The LLM Score has its own column in the output TSV file.

#### The JSON config file is updated to use "GPT-4_1106-preview" build.

Update 12-21-2023

available models on Ideker lab server:

| NAME           | ID           | SIZE   |
|----------------|--------------|--------|
| llama2:70b     | c3a7af098300 | 38 GB  |
| llama2:7b      | fe938a131f40 | 3.8 GB |
| llama2:latest  | fe938a131f40 | 3.8 GB |
| mistral:7b     | 4d9f4b269c33 | 4.1 GB |
| mixtral:latest | 99a9202f8a7a | 26 GB  |

In [3]:
import pandas as pd
import json 
from utils.openai_query import openai_chat
from utils.prompt_factory import make_user_prompt_with_score
from utils.server_model_query import server_model_chat
from utils.llm_analysis_utils import process_analysis, save_progress
from utils.genai_query import query_genai_model
from tqdm import tqdm
import constant
import openai
import os
import logging
import re
%load_ext autoreload

%autoreload 2


**Default run is using GPT4**

In [2]:
## load variables
initialize = True # if True, then initialize the input table with llm names, analysis and score to None 
# Replace with your actual values
config_file = './jsonFiles/toyexample.json'  # replace with your actual config file 
input_file = 'data/GO_term_analysis/toy_example_w_contaminated.csv' # replace with your actual input file
input_sep = ','  # replace with the separator
set_index = 'GO'  # replace with your column name that you want to set as index or None
gene_column = 'Genes'  # replace with your actual column name for the gene list
gene_sep = ' '  # replace with your actual separator
gene_features = None  # replace with your path to the gene features or None if you don't want to include in the prompt
direct = False # if True, then the prompt will be a direct sentence asking for a name and analysis from the gene set, otherwise default or customized prompt
out_file = 'data/GO_term_analysis/0120_heavychangeprompt_LLM_processed_toy_example_w_contamination_gpt_4'  # replace with your actual output file name

customized_prompt = False # if True, then the prompt will be the custom prompt, if False, then the prompt will use default

# load the config file
with open(config_file) as json_file:
    config = json.load(json_file)

if customized_prompt:
    # make sure the file exist 
    if os.path.isfile(config['CUSTOM_PROMPT_FILE']):
        with open(config['CUSTOM_PROMPT_FILE'], 'r') as f: # replace with your actual customized prompt file
            customized_prompt = f.read()
            assert len(customized_prompt) > 1, "Customized prompt is empty"
    else:
        print("Customized prompt file does not exist")
        customized_prompt = None
else:
    customized_prompt = None

# Load OpenAI key, context, and model used 
openai.api_key = os.environ["OPENAI_API_KEY"]

context = config['CONTEXT']
model = config['MODEL']
temperature = config['TEMP']
max_tokens = config['MAX_TOKENS']
if model.startswith('gpt'):
    rate_per_token = config['RATE_PER_TOKEN']
    DOLLAR_LIMIT = config['DOLLAR_LIMIT']
LOG_FILE = config['LOG_NAME']+'_log.json'

SEED = constant.SEED
column_prefix = model.split('-')[0]

In [3]:
# handle the logger so it create a new one for each model run
def get_logger(filename):
    logger = logging.getLogger(filename)
    logger.setLevel(logging.INFO)
    if not logger.handlers:
        file_handler = logging.FileHandler(filename)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)
    return logger


def main(df):
    analysis_dict  = {}

    logger = get_logger(f'{out_file}.log')

    i = 0 #used for track progress and saving the file
    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
        #only process None rows 
        if pd.notna(row[f'{column_prefix} Analysis']):
            continue
        
        gene_data = row[gene_column]
        # if gene_data is not a string, then skip
        if type(gene_data) != str:
            
            logger.warning(f'Gene set {idx} is not a string, skipping')
            continue
        genes = gene_data.split(gene_sep)
        
        if len(genes) >1000:
            logger.warning(f'Gene set {idx} is too big, skipping')
            continue

        try:
            prompt = make_user_prompt_with_score(genes)
            # print(prompt)
            finger_print = None
            if model.startswith('gpt'):
                print("Accessing OpenAI API")
                analysis, finger_print = openai_chat(context, prompt, model, temperature, max_tokens, rate_per_token, LOG_FILE, DOLLAR_LIMIT, SEED)
            elif model.startswith('gemini'):
                print("Using Google Gemini API")
                analysis, error_message = query_genai_model(f"{context}\n{prompt}", model, temperature, max_tokens, LOG_FILE) 
            else:
                print("Using server model")
                analysis, error_message= server_model_chat(context, prompt, model, temperature, max_tokens,LOG_FILE, SEED)

            
            if analysis:
                # print(analysis)
                llm_name, llm_score, llm_analysis = process_analysis(analysis)
                # clean up the score and return float
                try:
                    llm_score_value =  float(re.sub("[^0-9.-]", "", llm_score))
                except ValueError:
                    llm_score_value = llm_score
            
                
                df.loc[idx, f'{column_prefix} Name'] = llm_name
                df.loc[idx, f'{column_prefix} Analysis'] = llm_analysis
                df.loc[idx, f'{column_prefix} Score'] = llm_score_value
                analysis_dict[f'{idx}_{column_prefix}'] = analysis
                # Log success with fingerprint
                logger.info(f'Success for {idx} {column_prefix}.')
                if finger_print:
                    logger.info(f'GPT_Fingerprint for {idx}: {finger_print}')
                    
            else:
                logger.error(f'Error for query gene set {idx}: {error_message}')

        except Exception as e:
            logger.error(f'Error for {idx}: {e}')
            continue
        i += 1
        if i % 10 == 0:
            save_progress(df, analysis_dict, out_file)
            # df.to_csv(f'{out_file}.tsv', sep='\t', index=True)
            print(f"Saved progress for {i} genesets")
    # save the final file
    save_progress(df, analysis_dict, out_file)
    

In [4]:
#Define your own loop for running the pipeline
## 12-18-2023: this loop is for run the default gene set and the contaminated gene sets 
## can modify this loop for different models or only run on default gene set

##12-27-23: edited the prompt 
if __name__ == "__main__":
    
    df = pd.read_csv(input_file, sep=input_sep, index_col=set_index)
    
    if 'gpt' in model:
        name_fix = '_'.join(model.split('-')[:2])
    else:
        name_fix = model.replace(':', '_')
    column_prefix = name_fix + '_default'
    print(column_prefix)
    
    if initialize:
        # initialize the input file with llm names, analysis and score to None
        df[f'{column_prefix} Name'] = None
        df[f'{column_prefix} Analysis'] = None
        df[f'{column_prefix} Score'] = None
    main(df)  ## run with the real set 
    
    ## run the pipeline for contaiminated gene sets 
    contaminated_columns = [col for col in df.columns if col.endswith('contaminated_Genes')]
    # print(contaminated_columns)
    for col in contaminated_columns:
        gene_column = col ## Note need to change the gene_column to the contaminated column
        contam_prefix = '_'.join(col.split('_')[0:2])
        
        column_prefix = name_fix + '_' +contam_prefix
        print(column_prefix)

        if initialize:
            # initialize the input file with llm names, analysis and score to None
            df[f'{column_prefix} Name'] = None
            df[f'{column_prefix} Analysis'] = None
            df[f'{column_prefix} Score'] = None
        main(df)
    df.head()


gpt_4_default


  0%|          | 0/11 [00:00<?, ?it/s]

Accessing OpenAI API


  9%|▉         | 1/11 [00:21<03:36, 21.65s/it]

1761
Accessing OpenAI API


...

1475
Accessing OpenAI API


 91%|█████████ | 10/11 [04:58<00:28, 28.00s/it]

1423
Saved progress for 10 genesets
Accessing OpenAI API


100%|██████████| 11/11 [05:21<00:00, 29.25s/it]

1802


In [4]:
df = pd.read_csv(input_file, sep=input_sep, index_col=set_index)
df = df.loc['GO:2000136', :]

genes = df['Genes'].split(' ')
print(make_user_prompt_with_score(genes))


Write a critical analysis of the biological processes performed by this system of interacting proteins.
Base your analysis on prior knowledge available in your training data.
After completing your analysis, propose a brief and detailed name for the most prominent biological process performed by the system.
    
After completing your analysis, please also assign a confidence score to the process name you selected.
This score should follow the name in parentheses and range from 0.00 to 1.00. A score of 0.00 indicates the lowest confidence,
while 1.00 reflects the highest confidence. This score helps gauge how accurately the chosen name represents the functions and activities
within the system of interacting proteins. When determining your score, consider the proportion of genes in the protein system that participate
in the identified biological process. For instance, if you select "Ribosome biogenesis" as the process name but only a few genes in the system 
contribute to this process, t

In [3]:
import pandas as pd 
selected_go = pd.read_csv('data/GO_term_analysis/1000_selected_go_contaminated.csv')
# create a new dataframe by removing the 100 sets have been already ran 
model_compare_df = pd.read_csv('data/GO_term_analysis/model_compare/LLM_processed_model_compare_100set_gpt_4.tsv', sep='\t', index_col='GO')
model_compare_GO = model_compare_df.index.tolist()
print(len(model_compare_GO))

new = selected_go[~selected_go['GO'].isin(model_compare_GO)]
print(new.shape)
new.to_csv('data/GO_term_analysis/900_selected_go_contaminated.csv', index=False)

100
(900, 6)


In [9]:
## set up parameters for running gpt4 pipeline for the 1000 gene sets
import os 
from glob import glob
# Define start, step, and end values
start = 0
step = 50
end = 900 #already ran 100 before

# Create a range list
range_list = list(range(start, end + step, step))

# Create tuples for each consecutive pair in the list
tuple_list = [(range_list[i], range_list[i+1]) for i in range(len(range_list)-1)]


initialize = True 
input_file = './data/GO_term_analysis/900_selected_go_contaminated.csv'
input_sep = constant.GO_FILE_SEP
set_index = constant.GO_INDEX_COL  
gene_column = constant.GO_GENE_COL 
gene_sep = ' '

## create a param file 
conf_file = './jsonFiles/thousandGOrunGPT4_config.json'
params = []
for start, end in tuple_list:
    out_file = f'./data/GO_term_analysis/LLM_processed_gpt_4_{start}_{end}'  
    param = f"--config {conf_file} \
        --initialize \
        --input {input_file} \
        --input_sep  '{input_sep}'\
        --set_index {set_index} \
        --gene_column {gene_column}\
        --gene_sep '{gene_sep}' \
        --start {start} \
        --end {end} \
        --output_file {out_file}"
    print(param)
    params.append(param)
print('number of params: ', len(params))

with open('thousandGOsets_GPT4Run_params.txt', 'w') as f:
    for p in params:
        f.write(p+'\n')

--config ./jsonFiles/thousandGOrunGPT4_config.json         --initialize         --input ./data/GO_term_analysis/900_selected_go_contaminated.csv         --input_sep  ','        --set_index GO         --gene_column Genes        --gene_sep ' '         --start 0         --end 50         --output_file ./data/GO_term_analysis/LLM_processed_gpt_4_0_50
--config ./jsonFiles/thousandGOrunGPT4_config.json         --initialize         --input ./data/GO_term_analysis/900_selected_go_contaminated.csv         --input_sep  ','        --set_index GO         --gene_column Genes        --gene_sep ' '         --start 50         --end 100         --output_file ./data/GO_term_analysis/LLM_processed_gpt_4_50_100
--config ./jsonFiles/thousandGOrunGPT4_config.json         --initialize         --input ./data/GO_term_analysis/900_selected_go_contaminated.csv         --input_sep  ','        --set_index GO         --gene_column Genes        --gene_sep ' '         --start 100         --end 150         --output_fil

#### For CC and MF branch

In [9]:
## set up parameters for running gpt4 pipeline for the 1000 gene sets for CC and MF
import os 
from glob import glob
# Define start, step, and end values
start = 0
step = 50
end = 1000 


# Create a range list
range_list = list(range(start, end + step, step))

# Create tuples for each consecutive pair in the list
tuple_list = [(range_list[i], range_list[i+1]) for i in range(len(range_list)-1)]

input_sep = constant.GO_FILE_SEP
set_index = constant.GO_INDEX_COL  
gene_column = constant.GO_GENE_COL 
gene_sep = ' '

## create a param file 
conf_file = './jsonFiles/thousandGO_CC_MF_runGPT4_config.json'
params = []
for branch in ['CC', 'MF']:
    input_file = f'./data/GO_term_analysis/CC_MF_branch/{branch}_1000_selected_go_terms.csv'
    for start, end in tuple_list:
        out_file = f'./data/GO_term_analysis/CC_MF_branch/LLM_processed_{branch}terms_gpt_4_{start}_{end}'  
        param = f"--config {conf_file} \
            --initialize \
            --input {input_file} \
            --input_sep  '{input_sep}'\
            --set_index {set_index} \
            --gene_column {gene_column}\
            --gene_sep '{gene_sep}' \
            --start {start} \
            --end {end} \
            --output_file {out_file}"
        print(param)
        params.append(param)
print('number of params: ', len(params))

with open('thousandGOsets_CC_MF_GPT4Run_params.txt', 'w') as f:
    for p in params:
        f.write(p+'\n')

--config ./jsonFiles/thousandGO_CC_MF_runGPT4_config.json             --initialize             --input ./data/GO_term_analysis/CC_MF_branch/CC_1000_selected_go_terms.csv             --input_sep  ','            --set_index GO             --gene_column Genes            --gene_sep ' '             --start 0             --end 50             --output_file ./data/GO_term_analysis/CC_MF_branch/LLM_processed_CCterms_gpt_4_0_50
--config ./jsonFiles/thousandGO_CC_MF_runGPT4_config.json             --initialize             --input ./data/GO_term_analysis/CC_MF_branch/CC_1000_selected_go_terms.csv             --input_sep  ','            --set_index GO             --gene_column Genes            --gene_sep ' '             --start 50             --end 100             --output_file ./data/GO_term_analysis/CC_MF_branch/LLM_processed_CCterms_gpt_4_50_100
--config ./jsonFiles/thousandGO_CC_MF_runGPT4_config.json             --initialize             --input ./data/GO_term_analysis/CC_MF_branch/CC_1000_sele

In [1]:
# test the cost and time usage 
import json 
from glob import glob

logs = glob('./logs/thousand_GO_run_gpt4*.log')
total_cost = 0
total_run = 0
time_total = 0
for log in logs:
    with open(log, 'r') as f:
        data = json.load(f)
        total_cost += data['dollars_spent']
        total_run += data['runs']
        time_total += data['time_taken_total']
print('total cost: {:.2f}'.format(total_cost))
print('cost per run: {:.2f}'.format(total_cost/total_run))
print('time per run: {:.2f}'.format(time_total/total_run))



total cost: 43.75
cost per run: 0.05
time per run: 38.11


In [ ]:
# test the script for batch run

input_file = 'data/GO_term_analysis/toy_example.csv'
config = './jsonFiles/GOLLMrun_config.json'
set_index = 'GO'
gene_column = 'Genes'
gene_sep = ' '
start = 0
end = 5   
out_file = 'data/GO_term_analysis/LLM_processed_toy_example_gpt_4'
%run query_llm_for_analysis.py --config $config \
            --initialize \
            --input $input_file \
            --input_sep  ','\
            --set_index $set_index \
            --gene_column $gene_column\
            --gene_sep ' ' \
            --start $start \
            --end $end \
            --output_file $out_file

## Checkout and combine the output from the batch run 

In [22]:
from glob import glob
import pandas as pd
import json

### sanity check code along the way
processed_files = glob('data/GO_term_analysis/LLM_processed_gpt_4*.tsv')

for file in processed_files:
    df = pd.read_csv(file, sep='\t')
    df.set_index('GO', inplace=True)
    # check if the Analysis, Name and Score are all filled
    columns = [col for col in df.columns if col.endswith('Analysis') or col.endswith('Name') or col.endswith('Score')]
    print(columns)
    for col in columns:
        n_na = df[col].isna().sum()
        if n_na > 0:
            print(f'Error in {file} for {col}, has {n_na} NAs')
            print(df[df[col].isna()])
        else:
            continue
    # check if there is any duplicated GO terms
    print('Any duplicated GO: ',df.index.duplicated().sum())
    
    df.reset_index(inplace=True)
#     # print(ranges)
    print(df.shape)

    
combined_df = pd.concat([pd.read_csv(f, sep = '\t') for f in processed_files])
print(combined_df.shape)
print('Any duplicated GO: ',combined_df['GO'].duplicated().sum())
analysis_columns = [col for col in combined_df.columns if col.endswith('Analysis')]
print('Any duplicated LLM analysis: ', combined_df[analysis_columns[0]].duplicated(keep=False).sum())

combined_df.to_csv('data/GO_term_analysis/LLM_processed_selected_1000_go_terms.tsv', index=False, sep='\t')

['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
Any duplicated GO:  0
(50, 9)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
Any duplicated GO:  0
(50, 9)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
Any duplicated GO:  0
(50, 9)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
Any duplicated GO:  0
(50, 9)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
Any duplicated GO:  0
(50, 9)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
Any duplicated GO:  0
(50, 9)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
Any duplicated GO:  0
(50, 9)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
Any duplicated GO:  0
(50, 9)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
Any duplicated GO:  0
(50, 9)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
Any duplic

In [23]:
#combine with the 100 sets that are already ran 
model_compare_df = pd.read_csv('data/GO_term_analysis/model_compare/LLM_processed_model_compare_100set_gpt_4.tsv', sep='\t')
common_cols = [col for col in model_compare_df.columns if col in combined_df.columns]
print(common_cols)
model_compare_df = model_compare_df.loc[:, common_cols]
print(model_compare_df.shape)

combined_df = pd.concat([combined_df, model_compare_df])
print(combined_df.shape)
print('Any duplicated GO: ',combined_df['GO'].duplicated().sum())


['GO', 'Genes', 'Gene_Count', 'Term_Description', '50perc_contaminated_Genes', '100perc_contaminated_Genes', 'gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
(100, 9)
(1000, 9)
Any duplicated GO:  0


In [24]:

combined_df.to_csv('data/GO_term_analysis/LLM_processed_selected_1000_go_terms.tsv', index=False, sep='\t')

In [3]:
from glob import glob
import pandas as pd
import json

### sanity check code along the way

branches = ['CC', 'MF']
for branch in branches:
    branch_processed_files = glob(f'data/GO_term_analysis/CC_MF_branch/LLM_processed_{branch}terms_gpt_4*.tsv')
    print(len(branch_processed_files))
    for file in branch_processed_files:
        df = pd.read_csv(file, sep='\t')
        df.set_index('GO', inplace=True)
        # check if the Analysis, Name and Score are all filled
        columns = [col for col in df.columns if col.endswith('Analysis') or col.endswith('Name') or col.endswith('Score')]
        print(columns)
        for col in columns:
            n_na = df[col].isna().sum()
            if n_na > 0:
                print(f'Error in {file} for {col}, has {n_na} NAs')
                print(df[df[col].isna()])
            else:
                continue
        # check if there is any duplicated GO terms
        if df.index.duplicated().sum() > 0:
            print('Number of duplicated GO: ',df.index.duplicated().sum())
        
        df.reset_index(inplace=True)
    #     # print(ranges)
        print(df.shape)

        
    combined_df = pd.concat([pd.read_csv(f, sep = '\t') for f in branch_processed_files])
    print(combined_df.shape)
    print(f'Any duplicated GO in {branch} combined file: ',combined_df['GO'].duplicated().sum())
    analysis_columns = [col for col in combined_df.columns if col.endswith('Analysis')]
    print(f'Any duplicated {branch} LLM analysis: ', combined_df[analysis_columns[0]].duplicated(keep=False).sum())

    combined_df.to_csv(f'data/GO_term_analysis/CC_MF_branch/LLM_processed_selected_1000_go_{branch}terms.tsv', index=False, sep='\t')

20
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
(50, 7)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
(50, 7)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
(50, 7)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
(50, 7)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
(50, 7)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
(50, 7)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
(50, 7)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
(50, 7)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
(50, 7)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
(50, 7)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
(50, 7)
['gpt_4_default Name', 'gpt_4_default Analysis', 'gpt_4_default Score']
(50, 7)
['gpt_4_default Name', 'gpt_4_default